# 패스트텍스트(FastText)

reference https://wikidocs.net/22883

Word2Vec의 확장. Word2Vec와 다르게 단어안에 내부 단어(subword) 가 있다는 것으로 간주.

모르는 단어 , OOV 즉 Out of Vocabulary에 대한 대응이 어느 정도 가능해지고, 단어 집합내 빈도 수가 적었던 단어에 대한 대응(Rare Word)이 가능해진다. subword로 쪼갬으로써 오타가 섞인 단어들에 대해서도 일정 이상의 성능을 보인다.

# Word2Vec 와  FastText 비교

word2Vec는 해당 폴더의 eng_Word2Vec 실습에서 확인바람. 요약하자면, 데이터에 없는 단어를 넣으면 대응 할 수 없다는 결과가 나온다.

FastText 학습코드

In [1]:
import re
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize
from lxml import etree
from gensim.models import FastText

targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져옴
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통하여 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kwon_notebook\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\kwon_notebook\anaconda3\envs\tensorNN\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [5]:
model.wv.most_similar("supercharge")

[('charge', 0.8191316723823547),
 ('recharge', 0.8151326775550842),
 ('lockheed', 0.8146802186965942),
 ('superhydrophobic', 0.792341411113739),
 ('superhighway', 0.7838748097419739),
 ('discharge', 0.781169056892395),
 ('barge', 0.7804216742515564),
 ('gymnasium', 0.7764175534248352),
 ('cnc', 0.7752915024757385),
 ('ufo', 0.7739787697792053)]

word2vec는 학습하지 않은 단어에 대해서 유사한 단어를 찾아내지 못 했지만, FastText는 유사한 단어를 계산해서 출력하고 있음을 볼 수 있음